In [ ]:
#Librerias
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup

In [21]:
# Funcion get
def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    head = {'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) '
        'Chrome/59.0.3071.115 Safari/537.36'}
    try:
        with closing(get(url, stream=True, headers=head)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None


def is_good_response(resp):
    """
    Returns True if the response seems to be HTML, False otherwise.
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)


def log_error(e):
    """
    It is always a good idea to log errors. 
    This function just prints them, but you can
    make it do anything.
    """
    print(e)

In [22]:
#Raw HTML
raw_html = simple_get('https://howlongtobeat.com/game.php?id=10270')

In [ ]:
#Paso el request a un objeto de BeatifulSoup
html = BeautifulSoup(raw_html, 'html.parser')

# Parseo del HTML

## Titulo y rating

In [174]:
#Titulo del juego
titulo = html.find('div', {'class':"profile_header shadow_text"}).text.split('\n')
titulo = titulo[1].rstrip()

In [199]:
#Pasamos a dict
titulo_dict = {'Titulo' : titulo}
titulo_dict

{'Titulo': 'The Witcher 3: Wild Hunt'}

In [193]:
#Detalles de rating, backlog, etc
det=html.find('div', {'class':'profile_details'})

In [194]:
det

<div class="profile_details">
<li>2.1K Playing</li>
<li>7K Backlogs</li>
<li>911 Replays</li>
<li>1% Retired
</li>
<li>96% Rating</li>
<li>5.7K Beat</li>
<div class="clear"></div>
</div>

In [99]:
# Rating
details = []
for i, li in enumerate(det.select('li')):
        dicc = {i: li.text}
        details.append(li.text)
rating = details[4][0:2]

In [200]:
#Pasamos a dict
rating_dict = {'Rating': rating}
rating_dict

{'Rating': '96'}

## Completion Times

In [191]:
#Tiempos
tiempos = html.find_all('li', {'class':'short time_100'})

In [190]:
#Vemos como quedó
tiempos

[<li class="short time_100">
 <h5>Main Story</h5>
 <div>49½ Hours </div>
 </li>, <li class="short time_100">
 <h5>Main + Extras</h5>
 <div>103 Hours </div>
 </li>, <li class="short time_100">
 <h5>Completionist</h5>
 <div>171 Hours </div>
 </li>, <li class="short time_100">
 <h5>All Styles</h5>
 <div>102 Hours </div>
 </li>]

In [195]:
#Descompongo cada uno de los tiempos
times = []
for i in range(4):
    x = tiempos[i].select('div')
    x = str(x[0])
    x = x.replace('<div>', '')
    x = x.replace('</div>','')
    x = x.rstrip()
    times.append(x)

In [196]:
times

['49½ Hours', '103 Hours', '171 Hours', '102 Hours']

In [202]:
# Creo un diccionario con la clave de cada tiempo
times_dict = {'Main Story': times[0],
              'Main + Extras' : times[1],
              'Completionist' : times[2],
              'All Styles' : times[3]}
times_dict

{'All Styles': '102 Hours',
 'Completionist': '171 Hours',
 'Main + Extras': '103 Hours',
 'Main Story': '49½ Hours'}

## Informacion adicional del juego

In [32]:
# Info del juego
prof = html.find_all('div', {'class':'profile_info'})

[<div class="profile_info">
 <strong>
 Developer:
 </strong>
 CD Projekt RED </div>, <div class="profile_info">
 <strong>
 Publishers:
 </strong>
 CD Projekt, Warner Bros. Interactive Entertainment </div>, <div class="profile_info">
 <strong>
 Playable On:
 </strong>
 Linux, PC, PlayStation 4, Xbox One </div>, <div class="profile_info">
 <strong>
 Genres:
 </strong>
 Third-Person, Action, Adventure, Open World, Role-Playing </div>, <div class="profile_info">
 <strong>NA:</strong> May 19, 2015 </div>, <div class="profile_info">
 <strong>EU:</strong> May 19, 2015 </div>, <div class="profile_info">
 <strong>JP:</strong> May 19, 2015 </div>, <div class="profile_info">
 <strong>Updated:</strong> 7 Mins Ago </div>]

In [203]:
prof = html.find_all('div', {'class':'profile_info'})

In [207]:
# Creo un diccionario con la info
keys = ['Developer', 'Publishers', 'Playable On', 'Genres']
info_dict = {}

for i in range(4):
    
prof[0] ## Developer
prof[1] ## Publishers
prof[2] ## Playable On
prof[3] ##Genres

<div class="profile_info">
<strong>
Genres:
</strong>
Third-Person, Action, Adventure, Open World, Role-Playing </div>

In [219]:
prof.find

<div class="profile_info">
<strong>
Developer:
</strong>
CD Projekt RED </div>

In [ ]:
#Para juntar los diccionarios

{**titulo_dict, **rating_dict, **times_dict}